In [1]:
from kafka import KafkaConsumer
from json import loads

import ewtpy
from skimage.transform import resize
import numpy as np
import pandas as pd

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
def get_Ewt(signal, p):
    ewt,  mfb ,boundaries = ewtpy.EWT1D(signal, N = p)
    return ewt

In [4]:
# consumers to get the vobration signals
consumer_horizontal = KafkaConsumer('phm-x',
                         bootstrap_servers=['59.124.112.101:32110'],
                         value_deserializer= lambda x: loads(x.decode('utf-8')))

consumer_vertical = KafkaConsumer('phm-y',
                         bootstrap_servers=['59.124.112.101:32110'],
                         value_deserializer= lambda x: loads(x.decode('utf-8')))

In [5]:
from influxdb import InfluxDBClient

def get_influx_client():
    host ='influxdb.motorflow.iii-ei-stack.com'
    port=443
    dbName= 'motorflow-phm'
    user= 'admin'
    pwd='2An3rgRVRj'
    client =  InfluxDBClient(host, port,user, pwd, dbName, ssl=True, verify_ssl=True)
    return client

In [6]:
def push_influx(msg, measurement):
    influx_client = get_influx_client()
    data ={}
    data['measurement']= measurement
    fields ={}
    for i in range(0, len(msg)):
        fields['f'+str(i)] = msg[i]
    data['fields']= fields
    json_body = []
    json_body.append(data)
#    print('JSON BODY', json_body)
    written = influx_client.write_points(json_body)
#    print('value of writeeeeeee', written)
    if written == True:
        print('successfully written to DB')
    else :
        print('Could not write into DB')

In [7]:
def push_result_influx(HI_value, rul_time_seconds, rul_time_hours, degredation_percentage,
                       rul_percentage, measurement):
    influx_client = get_influx_client()
    data ={}
    data['measurement']= measurement
    fields ={}
    fields['HI value'] = HI_value
    fields['RUL (Seconds)'] = rul_time_seconds
    fields['RUL (Hours)'] = rul_time_hours
    fields['Degredation Percenatge'] = degredation_percentage
    fields['RUL Percenatge'] = rul_percentage
    data['fields']= fields
    json_body = []
    json_body.append(data)
#    print('JSON BODY', json_body)
    written = influx_client.write_points(json_body)
#    print('value of writeeeeeee', written)
    if written == True:
        pass
#         print('successfully written the results to DB')
    else :
        print('Could not write the results into DB')

In [8]:
image_size =(128, 128)
p=5

failureTime = 28030 # failure time for bearing 1_1 in seconds.

In [9]:
# load the prediction model 
from keras.models import load_model
pred_model = load_model('model/CNNHI_bearing1_1.h5')

Using TensorFlow backend.


In [20]:
for (h, v) in zip (consumer_horizontal, consumer_vertical):
    
    #initializing test datset
    test_data = np.zeros((1, 128, 128))
    
    #reading index value for timestamp
    sample_time = h.value[0] * 10       #time in seconds (10 secinds whole data)
#     print('time in seconds : ', sample_time )
    
    ######## fucntion call to push to 1 second data into influx db #############
#     push_influx(h.value[1:257], 'phm_horizontal')
#     push_influx(v.value[1:257], 'phm_vertical')
    
    ###########################
    # preparing the dataframe to store collected vibration signals
    df = pd.DataFrame()
    df['horizontal'] = h.value[1:]      #avoiding index value
    df['vertical']   = v.value[1:]      #avoiding index value
    td = np.asarray(df).reshape(-1)
    
    #calculating EWT spectrum
    data = resize(get_Ewt(td, p), image_size, mode= 'constant')
    test_data[0, :, :] = data
    ds_test = test_data.reshape(test_data.shape[0], 1, 128, 128)
    ds_test = ds_test.astype('float32')
    
    #Predictiing the HI value for the given inuput signal
    pred = pred_model.predict(ds_test)
    HI_value =pred[0][0] 
    print('Health Indicator Value : %.2f' % HI_value)
    
    rul_time_seconds = failureTime - sample_time        # Remaining life time of the motor in seconds
    print('RUL time in (Seconds) : %.2f' % rul_time_seconds)
    
    rul_time_hours = rul_time_seconds/3600.0            # Remaining life time of the motor in hours
    print('RUL time in (Hours) : %.2f' % rul_time_hours)
    
    degredation_percentage = (sample_time *100)/float(failureTime)
#     print('Degredation Percentage : %.2f ' % degredation_percentage)
    
    rul_percentage = 100- degredation_percentage
    print('RUL Percentage : %.2f ' % rul_percentage)
    
    print('\n')
    print('####################################')
    
    
    ############ fucntion call to push to result data into influx db 
#     push_result_influx(HI_value, rul_time_seconds, rul_time_hours, degredation_percentage,
#                        rul_percentage, 'results_bearing1_1')

Health Indicator Value : 0.56
RUL time in (Seconds) : 22910.00
RUL time in (Hours) : 6.36
RUL Percentage : 81.73 


####################################
Health Indicator Value : 0.52
RUL time in (Seconds) : 22900.00
RUL time in (Hours) : 6.36
RUL Percentage : 81.70 


####################################
Health Indicator Value : 0.55
RUL time in (Seconds) : 22890.00
RUL time in (Hours) : 6.36
RUL Percentage : 81.66 


####################################
Health Indicator Value : 0.48
RUL time in (Seconds) : 22880.00
RUL time in (Hours) : 6.36
RUL Percentage : 81.63 


####################################
Health Indicator Value : 0.57
RUL time in (Seconds) : 22870.00
RUL time in (Hours) : 6.35
RUL Percentage : 81.59 


####################################
Health Indicator Value : 0.60
RUL time in (Seconds) : 22860.00
RUL time in (Hours) : 6.35
RUL Percentage : 81.56 


####################################
Health Indicator Value : 0.58
RUL time in (Seconds) : 22850.00
RUL time in (Hours)

Health Indicator Value : 0.47
RUL time in (Seconds) : 22360.00
RUL time in (Hours) : 6.21
RUL Percentage : 79.77 


####################################
Health Indicator Value : 0.47
RUL time in (Seconds) : 22350.00
RUL time in (Hours) : 6.21
RUL Percentage : 79.74 


####################################
Health Indicator Value : 0.52
RUL time in (Seconds) : 22340.00
RUL time in (Hours) : 6.21
RUL Percentage : 79.70 


####################################
Health Indicator Value : 0.52
RUL time in (Seconds) : 22330.00
RUL time in (Hours) : 6.20
RUL Percentage : 79.66 


####################################
Health Indicator Value : 0.45
RUL time in (Seconds) : 22320.00
RUL time in (Hours) : 6.20
RUL Percentage : 79.63 


####################################
Health Indicator Value : 0.48
RUL time in (Seconds) : 22310.00
RUL time in (Hours) : 6.20
RUL Percentage : 79.59 


####################################
Health Indicator Value : 0.52
RUL time in (Seconds) : 22300.00
RUL time in (Hours)

Health Indicator Value : 0.51
RUL time in (Seconds) : 21820.00
RUL time in (Hours) : 6.06
RUL Percentage : 77.85 


####################################
Health Indicator Value : 0.52
RUL time in (Seconds) : 21810.00
RUL time in (Hours) : 6.06
RUL Percentage : 77.81 


####################################
Health Indicator Value : 0.55
RUL time in (Seconds) : 21800.00
RUL time in (Hours) : 6.06
RUL Percentage : 77.77 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 21790.00
RUL time in (Hours) : 6.05
RUL Percentage : 77.74 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 21780.00
RUL time in (Hours) : 6.05
RUL Percentage : 77.70 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 21770.00
RUL time in (Hours) : 6.05
RUL Percentage : 77.67 


####################################
Health Indicator Value : 0.50
RUL time in (Seconds) : 21760.00
RUL time in (Hours)

Health Indicator Value : 0.56
RUL time in (Seconds) : 21280.00
RUL time in (Hours) : 5.91
RUL Percentage : 75.92 


####################################
Health Indicator Value : 0.59
RUL time in (Seconds) : 21270.00
RUL time in (Hours) : 5.91
RUL Percentage : 75.88 


####################################
Health Indicator Value : 0.62
RUL time in (Seconds) : 21260.00
RUL time in (Hours) : 5.91
RUL Percentage : 75.85 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 21250.00
RUL time in (Hours) : 5.90
RUL Percentage : 75.81 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 21240.00
RUL time in (Hours) : 5.90
RUL Percentage : 75.78 


####################################
Health Indicator Value : 0.57
RUL time in (Seconds) : 21230.00
RUL time in (Hours) : 5.90
RUL Percentage : 75.74 


####################################
Health Indicator Value : 0.59
RUL time in (Seconds) : 21220.00
RUL time in (Hours)

Health Indicator Value : 0.65
RUL time in (Seconds) : 20740.00
RUL time in (Hours) : 5.76
RUL Percentage : 73.99 


####################################
Health Indicator Value : 0.63
RUL time in (Seconds) : 20730.00
RUL time in (Hours) : 5.76
RUL Percentage : 73.96 


####################################
Health Indicator Value : 0.56
RUL time in (Seconds) : 20720.00
RUL time in (Hours) : 5.76
RUL Percentage : 73.92 


####################################
Health Indicator Value : 0.60
RUL time in (Seconds) : 20710.00
RUL time in (Hours) : 5.75
RUL Percentage : 73.89 


####################################
Health Indicator Value : 0.58
RUL time in (Seconds) : 20700.00
RUL time in (Hours) : 5.75
RUL Percentage : 73.85 


####################################
Health Indicator Value : 0.55
RUL time in (Seconds) : 20690.00
RUL time in (Hours) : 5.75
RUL Percentage : 73.81 


####################################
Health Indicator Value : 0.58
RUL time in (Seconds) : 20680.00
RUL time in (Hours)

...

Health Indicator Value : 0.65
RUL time in (Seconds) : 16870.00
RUL time in (Hours) : 4.69
RUL Percentage : 60.19 


####################################
Health Indicator Value : 0.59
RUL time in (Seconds) : 16860.00
RUL time in (Hours) : 4.68
RUL Percentage : 60.15 


####################################
Health Indicator Value : 0.58
RUL time in (Seconds) : 16850.00
RUL time in (Hours) : 4.68
RUL Percentage : 60.11 


####################################
Health Indicator Value : 0.62
RUL time in (Seconds) : 16840.00
RUL time in (Hours) : 4.68
RUL Percentage : 60.08 


####################################
Health Indicator Value : 0.60
RUL time in (Seconds) : 16830.00
RUL time in (Hours) : 4.67
RUL Percentage : 60.04 


####################################
Health Indicator Value : 0.65
RUL time in (Seconds) : 16820.00
RUL time in (Hours) : 4.67
RUL Percentage : 60.01 


####################################
Health Indicator Value : 0.62
RUL time in (Seconds) : 16810.00
RUL time in (Hours)

Health Indicator Value : 0.49
RUL time in (Seconds) : 27710.00
RUL time in (Hours) : 7.70
RUL Percentage : 98.86 


####################################
Health Indicator Value : 0.42
RUL time in (Seconds) : 27700.00
RUL time in (Hours) : 7.69
RUL Percentage : 98.82 


####################################
Health Indicator Value : 0.44
RUL time in (Seconds) : 27690.00
RUL time in (Hours) : 7.69
RUL Percentage : 98.79 


####################################
Health Indicator Value : 0.35
RUL time in (Seconds) : 27680.00
RUL time in (Hours) : 7.69
RUL Percentage : 98.75 


####################################
Health Indicator Value : 0.43
RUL time in (Seconds) : 27670.00
RUL time in (Hours) : 7.69
RUL Percentage : 98.72 


####################################
Health Indicator Value : 0.37
RUL time in (Seconds) : 27660.00
RUL time in (Hours) : 7.68
RUL Percentage : 98.68 


####################################
Health Indicator Value : 0.26
RUL time in (Seconds) : 27650.00
RUL time in (Hours)

Health Indicator Value : 0.40
RUL time in (Seconds) : 27160.00
RUL time in (Hours) : 7.54
RUL Percentage : 96.90 


####################################
Health Indicator Value : 0.31
RUL time in (Seconds) : 27150.00
RUL time in (Hours) : 7.54
RUL Percentage : 96.86 


####################################
Health Indicator Value : 0.30
RUL time in (Seconds) : 27140.00
RUL time in (Hours) : 7.54
RUL Percentage : 96.82 


####################################
Health Indicator Value : 0.39
RUL time in (Seconds) : 27130.00
RUL time in (Hours) : 7.54
RUL Percentage : 96.79 


####################################
Health Indicator Value : 0.34
RUL time in (Seconds) : 27120.00
RUL time in (Hours) : 7.53
RUL Percentage : 96.75 


####################################
Health Indicator Value : 0.37
RUL time in (Seconds) : 27110.00
RUL time in (Hours) : 7.53
RUL Percentage : 96.72 


####################################
Health Indicator Value : 0.42
RUL time in (Seconds) : 27100.00
RUL time in (Hours)

Health Indicator Value : 0.30
RUL time in (Seconds) : 26620.00
RUL time in (Hours) : 7.39
RUL Percentage : 94.97 


####################################
Health Indicator Value : 0.30
RUL time in (Seconds) : 26610.00
RUL time in (Hours) : 7.39
RUL Percentage : 94.93 


####################################
Health Indicator Value : 0.30
RUL time in (Seconds) : 26600.00
RUL time in (Hours) : 7.39
RUL Percentage : 94.90 


####################################
Health Indicator Value : 0.36
RUL time in (Seconds) : 26590.00
RUL time in (Hours) : 7.39
RUL Percentage : 94.86 


####################################
Health Indicator Value : 0.28
RUL time in (Seconds) : 26580.00
RUL time in (Hours) : 7.38
RUL Percentage : 94.83 


####################################
Health Indicator Value : 0.29
RUL time in (Seconds) : 26570.00
RUL time in (Hours) : 7.38
RUL Percentage : 94.79 


####################################
Health Indicator Value : 0.32
RUL time in (Seconds) : 26560.00
RUL time in (Hours)

KeyboardInterrupt: 

In [10]:
def phm_inference(data):
    config = ConfigProto()
    config.gpu_options.allow_growth = True
    session = InteractiveSession(config=config)
    model = pred_model
    image_size = (128, 128)
    p = 5
    failureTime = 28030 # failure time for bearing 1_1 in seconds.
    #initializing test datset
    sample_time = data["x"][0] * 10
    test_data = np.zeros((1, 128, 128))
    df = pd.DataFrame()
    df['horizontal'] = data["x"][1:]      #avoiding index value
    df['vertical']   = data["y"][1:]      #avoiding index value
    td = np.asarray(df).reshape(-1)

    #calculating EWT spectrum
    data = resize(get_Ewt(td, p), image_size, mode= 'constant')
    test_data[0, :, :] = data
    ds_test = test_data.reshape(test_data.shape[0], 1, 128, 128)
    ds_test = ds_test.astype('float32')

    #Predictiing the HI value for the given inuput signal
    pred = model.predict(ds_test)
    HI_value = pred[0][0] 
    rul_time_seconds = failureTime - sample_time        # Remaining life time of the motor in seconds
    rul_time_hours = rul_time_seconds/3600.0            # Remaining life time of the motor in hours
    degredation_percentage = (sample_time *100)/float(failureTime)
    rul_percentage = 100- degredation_percentage

    pred_class = {
        "HI_value": HI_value,
        "rul_time_seconds": rul_time_seconds,
        "rul_time_hours": rul_time_hours,
        "degredation_percentage": degredation_percentage,
        "rul_percentage": rul_percentage
    }
    return pred_class

In [ ]:
while (True):
    for (h, v) in zip (consumer_horizontal, consumer_vertical):
        data = {"x": h.value, "y":v.value}
        result = phm_inference(data)
    #     print(result)
        push_result_influx(
            result["HI_value"],
            result["rul_time_seconds"],
            result["rul_time_hours"],
            result["degredation_percentage"],
            result["rul_percentage"],
            'results_bearing1_1'
        )


/root/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
